In [41]:
from confluent_kafka import Consumer 
import json
from queue import Queue
import numpy as np 
import pandas as pd

In [42]:
def get_mean(x):
    return np.array(x).mean()

In [43]:
def consume_rally_data(topic, batch_size):
    conf = {
        'bootstrap.servers':'localhost:9092',
        'group.id':'mygroup',
        'auto.offset.reset':'earliest'
    }

    consumer = Consumer(conf)
    consumer.subscribe([topic])
    buffer = []
    return_data = []

   
    for _ in range(batch_size):
        msg = consumer.poll(1.0)
        if msg is None: 
            continue 
        if msg.error():
            print(f'Consumer Error: {msg.error()}')
            continue

        data = json.loads(msg.value().decode('utf-8'))
        buffer.append(data)

        if len(buffer) == batch_size:
            
            return_data.extend(buffer)
            buffer.clear()

    consumer.close()
    return json.dumps(return_data)


In [44]:
def Rally_Data_Analysis(data_queue):
    data = json.loads(data_queue)
    df = pd.json_normalize(data, errors = 'ignore')
    
    headers = [
            'vehicle performance.engine_rpm', 'vehicle performance.engine_temp',
            'vehicle performance.fuel_level', 'vehicle performance.oil_pressure',
            'vehicle performance.suspension_travel.FL',
            'vehicle performance.suspension_travel.FR',
            'vehicle performance.suspension_travel.RL',
            'vehicle performance.suspension_travel.RR',
            'vehicle performance.tire_pressure.FL',
            'vehicle performance.tire_pressure.FR',
            'vehicle performance.tire_pressure.RL',
            'vehicle performance.tire_pressure.RR',
            'vehicle performance.transmission_temp', 'weather.barometric_pressure',
            'weather.external_temp', 'weather.humidity'
        ]
    
    df1 = df.copy()
    for h in headers:
        df1[h] = df1[h].apply(lambda x: get_mean(x))

    df2 = df1[headers]
    df2 = df2.mean()

    df2['location.altitude'] = df1.iloc[-1]['location.altitude'][2]
    df2['location.compass_heading'] = df1.iloc[-1]['location.compass_heading'][2]
    df2['location.gps_coordinates'] = df1.iloc[-1]['location.gps_coordinates'][2]
    df2['time stamp'] = df1.iloc[-1]['time stamp']

    return df2.to_json() 

In [45]:
data_queue = consume_rally_data('dakar_rally_sim', 5)

%5|1738065278.840|REQTMOUT|rdkafka#consumer-8| [thrd:GroupCoordinator]: GroupCoordinator/0: Timed out LeaveGroupRequest in flight (after 5004ms, timeout #0): possibly held back by preceeding blocking JoinGroupRequest with timeout in 292996ms
%4|1738065278.840|REQTMOUT|rdkafka#consumer-8| [thrd:GroupCoordinator]: GroupCoordinator/0: Timed out 1 in-flight, 0 retry-queued, 0 out-queue, 0 partially-sent requests
%3|1738065278.840|FAIL|rdkafka#consumer-8| [thrd:GroupCoordinator]: GroupCoordinator: abdul-Precision-7540:9092: 1 request(s) timed out: disconnect (average rtt 0.270ms) (after 10004ms in state UP)


In [46]:
data = Rally_Data_Analysis(data_queue)
data = json.loads(data)
data

KeyError: 'vehicle performance.engine_rpm'

In [32]:
data['vehicle performance.engine_rpm']

4107.160815753